In [1]:
import cv2
import base64
import numpy as np
import matplotlib.pyplot as plt

## 1. Image Preprocessing
---

In [2]:
# Image Preprocessing Params
blurr_kernel_size = 5
risize_factor = 2

# Line Approximation Params
fixed_epsilon = 1.0

use_dynamic_epsilon = False
epsilon_factor = 0.02
max_epsilon = 2.5
min_epsilon = 1.5

# GCODE Params
z_safe_hight = 10.0
z_working_hight = 0.5
z_depth = 3
z_feed = 500
xy_feed = 1000
spindle_speed = 24000
# Maximaler Vorschub ist ca. 3000 (immer bei G0) (3000mm pro Minute)

In [3]:
# Custom Image
edge_image_custom = np.array([
       [  0,   0,   0,   0,   0,   0,   0, 255,   0,   0],
       [  0,   0,   0,   0,   0,   0, 255,   0,   0,   0],
       [  0,   0,   0, 255,   0, 255,   0,   0,   0,   0],
       [  0,   0,   0, 255,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 255,   0,   0,   0,   0,   0, 255],
       [  0,   0,   0, 255,   0,   0,   0,   0, 255,   0],
       [  0,   0,   0,   0,   0,   0,   0, 255,   0,   0],
       [  0,   0,   0,   0,   0,   0, 255, 255, 255,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255]]).astype(np.uint8)

# Preloaded Image
preloaded_images = None
with open("../preloaded_images.txt", "r") as file:
    preloaded_images = eval(file.read())
selected_diff_image = preloaded_images[2]

# Convert image
decoded_image = base64.b64decode(selected_diff_image.split(',')[1])
image_array = np.frombuffer(decoded_image, dtype=np.uint8)
origial = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
gray = cv2.cvtColor(origial, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (blurr_kernel_size, blurr_kernel_size), 0)
edges = cv2.Canny(blurred, 100, 200)
edge_image_preloaded = cv2.resize(edges, (edges.shape[1] // risize_factor, edges.shape[0] // risize_factor))

## 1. Edge Approximation
---

In [4]:
def getEdgeApprox(edge_image):
       contours, _ = cv2.findContours(edge_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
       edge_approximations = []

       for i in range(len(contours)):
              edge_approx = cv2.approxPolyDP(contours[i], fixed_epsilon, True)
              
              # remove last element if it is already in contour
              if edge_approx[-1] in edge_approx[:len(edge_approx)-2]:
                     edge_approx = edge_approx[:-1]
              
              edge_approximations.append(edge_approx)
       
       return edge_approximations


def plotEdgeApprox(edge_image, edge_approximations, markersize):
       # Plot des Edge-Bildes
       plt.imshow(edge_image, cmap='gray')
       for edge_approx in edge_approximations:
              x_approx = edge_approx[:, :, 0].flatten()
              y_approx = edge_approx[:, :, 1].flatten()
              plt.plot(x_approx, y_approx, color='blue')
              plt.plot(x_approx, y_approx, 'ro', markersize=markersize)

       plt.title('Edge Approximation')
       plt.axis('scaled')
       plt.show()

In [5]:
# custom edges
edge_approximations_custom = getEdgeApprox(edge_image_custom)
# plotEdgeApprox(edge_image_custom, edge_approximations_custom, 5)

edge_approximations_preloaded = getEdgeApprox(edge_image_preloaded)
# plotEdgeApprox(edge_image_preloaded, edge_approximations_preloaded, 1.5)

## 2. Shortest Path
---

In [6]:
# Calculate the Distance between contours (to be minimized)
def calculate_g0_distance(contours):
    total_distance = 0
    start_end_point = np.array([0, 0])
    recent_contour_end = start_end_point

    for contour in contours:
        contour_points = contour.squeeze()
        start = contour_points[0] # set startPoint of Contour
        total_distance += np.linalg.norm(start - recent_contour_end)  # Calc euklidian dinstance
        recent_contour_end = contour_points[-1] # set endPoint of Contour

    total_distance += np.linalg.norm(np.array([0, 0]) - recent_contour_end)

    return total_distance

### 2.1 Heuristic Algorithm (Nearest Neighbor)
---
- Pros: You get relatively fast a good result
- Cons: You will not find very good paths, because the algorithm uses only the current and the last contour

In [7]:
from scipy.spatial.distance import cdist

def getContourStartPoint(contours):
    contour_start_points = []
    contour_mapping = []
    for i, contour in enumerate(contours):
        contour_start_points.append(np.array(contour[0][0])) # start_point
        contour_start_points.append(np.array(contour[-1][0])) # end_point

        contour_mapping.append((i, False)) # start_point
        contour_mapping.append((i, True)) # end_point

    return np.array(contour_start_points), np.array(contour_mapping)

start_point = np.array([0, 0])

def optimize_contour_order(contours):
    contour_start_points, contour_mapping = getContourStartPoint(contours) 

    new_order = np.array([])
    contour_end_point = start_point # set the starting point as fist point

    while True:
        nn_index = np.argmin(cdist([contour_end_point], contour_start_points)) # get nearest neighbor index
        nn_p_index = nn_index + 1 if nn_index % 2 == 0 else nn_index - 1 # get partner index
        contour_end_point = contour_start_points[nn_p_index] # Set new contour end

        new_order = np.append(new_order, contour_mapping[nn_index], axis=0) # Update Order

        contour_start_points = np.delete(contour_start_points, [nn_index, nn_p_index], axis=0) # Remove used Kontours
        contour_mapping = np.delete(contour_mapping, [nn_index, nn_p_index], axis=0) # Remove used Kontours

        if len(contour_start_points) == 0:
            break

    # use new_order for contour and reverse some contours
    return [contours[int(c_task[0])] if int(c_task[1]) == 0 else contours[int(c_task[0])][::-1] for c_task in new_order.reshape((-1, 2))] 

In [8]:
import time

start_time = time.time()
ordered_contours = optimize_contour_order(edge_approximations_preloaded)
duration = time.time() - start_time

# Plot Improvement
print('original', calculate_g0_distance(edge_approximations_preloaded))
print('reordered', calculate_g0_distance(ordered_contours))
print('calc time', duration)

original 75633.97699643619
reordered 6801.493595273359
calc time 0.18583202362060547


## 4. Generate GCODE
---

In [9]:
def generateGCODE(contours):
    gcode_lines = []

    # Drehgeschwindigkeit und initiale Höhe festlegen
    gcode_lines += [
        f'M03 S{spindle_speed}', 
        f'G00 Z{z_safe_hight}'
    ]

    # GCODE für die Kontouren
    for i, edge_approx in enumerate(contours):
        gcode_lines += [
            f'######## Contour {i+1} ########',
            f'G00 X{edge_approx[0][0][0]} Y{edge_approx[0][0][1]}',
            f'G00 Z{z_working_hight}' if i == 0 else None,
            'G00 Z0',
            f'G01 Z-3 F{z_feed}',
            f'G01 X{edge_approx[1][0][0]} Y{edge_approx[1][0][1]} F{xy_feed}' if len(edge_approx) > 1 else None,
            *[f'G01 X{edge[0][0]} Y{edge[0][1]}' for edge in edge_approx[2:]],
            f'G00 Z{z_working_hight}'
        ]

    # Fräßkopf zu initialer Position zurückbewegen
    gcode_lines += [
        '######## End ########',
        f'G00 Z{z_safe_hight}',
        'G00 X0 Y0',
        'M05',
        'M30'
    ]

    # None Elemente entfernen und GCODE erstellen
    gcode = '\n'.join([line for line in gcode_lines if line != None])

    # Save the G-code to a file
    with open('gcode.tap', "w") as f:
        f.writelines(gcode)

    # Print GCODE Stats
    print('\n'.join([
        'Generated GCODE',
        f'Amount of Contours: {len(contours)}',
        f'Amount of Lines: {len(gcode_lines)}',
    ]))

In [10]:
generateGCODE(ordered_contours)

Generated GCODE
Amount of Contours: 1426
Amount of Lines: 12668


In [11]:
def getContourStartPoint(contours):
    contour_start_points = []
    contour_mapping = []
    for i, contour in enumerate(contours):
        contour_start_points.append(np.array(contour[0][0])) # start_point
        contour_start_points.append(np.array(contour[-1][0])) # end_point

        contour_mapping.append((i, False)) # start_point
        contour_mapping.append((i, True)) # end_point

    return np.array(contour_start_points), np.array(contour_mapping)

In [12]:
contour_start_points, contour_mapping = getContourStartPoint(contours)

NameError: name 'contours' is not defined

In [ ]:
contour_mapping

array([[1, 0],
       [1, 1],
       [2, 0],
       [2, 1],
       [3, 0],
       [3, 1]])

In [ ]:
a = np.array([[]])
a = np.append(a, contour_mapping[0], axis=1)
a = np.append(a, contour_mapping[1], axis=1)
print(a)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:
def getContourStartPoint(contours):
    contour_start_points = []
    contour_mapping = []
    for i, contour in enumerate(contours):
        contour_start_points.append(np.array(contour[0][0])) # start_point
        contour_start_points.append(np.array(contour[-1][0])) # end_point

        contour_mapping.append((i, False)) # start_point
        contour_mapping.append((i, True)) # end_point

    return np.array(contour_start_points), np.array(contour_mapping)

start_point = np.array([0, 0])

def optimize_contour_order(contours):
    contour_start_points, contour_mapping = getContourStartPoint(contours) 

    new_order = np.array([])
    contour_end_point = start_point # set the starting point as fist point

    while True:
        nn_index = np.argmin(cdist([contour_end_point], contour_start_points)) # get nearest neighbor index
        nn_p_index = nn_index + 1 if nn_index % 2 == 0 else nn_index - 1 # get partner index
        contour_end_point = contour_start_points[nn_p_index] # Set new contour end

        new_order = np.append(new_order, contour_mapping[nn_index], axis=0) # Update Order

        contour_start_points = np.delete(contour_start_points, [nn_index, nn_p_index], axis=0) # Remove used Kontours
        contour_mapping = np.delete(contour_mapping, [nn_index, nn_p_index], axis=0) # Remove used Kontours

        if len(contour_start_points) == 0:
            break

    # use new_order for contour and reverse some contours
    return [contours[int(c_task[0])] if int(c_task[1]) == 0 else contours[int(c_task[0])][::-1] for c_task in new_order.reshape((-1, 2))] 

In [ ]:
edge_approximations_custom

contour_start_points = [[9, 4], [7, 6], [3, 2], [3, 5], [7, 0], [5, 2]]

contour_mapping = {
    '[9, 4]': (1, False), # (contourIndex, reversedContour)
    '[7, 6]': (1, True),
    '[3, 2]': (2, False),
    '[3, 5]': (2, True),
    '[7, 0]': (3, False),
    '[5, 2]': (3, True)
}



[array([[[9, 4]],
 
        [[6, 7]],
 
        [[8, 7]],
 
        [[9, 9]],
 
        [[7, 6]]], dtype=int32),
 array([[[3, 2]],
 
        [[3, 5]]], dtype=int32),
 array([[[7, 0]],
 
        [[5, 2]]], dtype=int32)]

In [ ]:
import time

start_time = time.time()
ordered_contours = optimize_contour_order(edge_approximations_custom)
duration = time.time() - start_time

# Plot Improvement
print('original', calculate_g0_distance(edge_approximations_custom))
print('reordered', calculate_g0_distance(ordered_contours))
print('calc time', duration)

[array([[[9, 4]],

       [[6, 7]]], dtype=int32), array([[[3, 2]],

       [[3, 5]]], dtype=int32), array([[[7, 0]],

       [[5, 2]]], dtype=int32)]
original 27.46809874120876
reordered 0.0
calc time 0.0006029605865478516


In [ ]:
import time

start_time = time.time()
ordered_contours = optimize_contour_order(edge_approximations_preloaded[:300])
duration = time.time() - start_time

# Plot Improvement
print('original', calculate_g0_distance(edge_approximations_preloaded))
print('reordered', calculate_g0_distance(ordered_contours))
print('calc time', duration)

original 106222.96129783994
reordered 21107.011432987747
calc time 5.855712175369263


In [ ]:
len(edge_approximations_preloaded)

1426

## 4. Generate GCODE
---

In [ ]:
gcode_lines = []

# Drehgeschwindigkeit und initiale Höhe festlegen
gcode_lines += [
    f'M03 S{spindle_speed}', 
    f'G00 Z{z_safe_hight}'
]

# GCODE für die Kontouren
for i, edge_approx in enumerate(edge_approximations):
    gcode_lines += [
        f'######## Contour {i+1} ########',
        f'G00 X{edge_approx[0][0][0]} Y{edge_approx[0][0][1]}',
        f'G00 Z{z_working_hight}' if i == 0 else None,
        'G00 Z0',
        f'G01 Z-3 F{z_feed}',
        f'G01 X{edge_approx[1][0][0]} Y{edge_approx[1][0][1]} F{xy_feed}' if len(edge_approx) > 1 else None,
        *[f'G01 X{edge[0][0]} Y{edge[0][1]}' for edge in edge_approx[2:]],
        f'G00 Z{z_working_hight}'
    ]

# Fräßkopf zu initialer Position zurückbewegen
gcode_lines += [
    '######## End ########',
    f'G00 Z{z_safe_hight}',
    'G00 X0 Y0',
    'M05',
    'M30'
]

# None Elemente entfernen und GCODE erstellen
gcode = '\n'.join([line for line in gcode_lines if line != None])

# Save the G-code to a file
with open('gcode.tap', "w") as f:
    f.writelines(gcode)

# Print GCODE Stats
gcode_stats = [
    'Generated GCODE',
    f'Amount of Contours: {len(edge_approximations)}',
    f'Amount of Lines: {len(gcode_lines)}',
]

print('\n'.join(gcode_stats))

Generated GCODE
Amount of Contours: 3
Amount of Lines: 30


In [ ]:
edge_approximations[0][0:]

array([[[8, 5]],

       [[5, 8]],

       [[8, 9]],

       [[6, 7]]], dtype=int32)